# Retail Stores

In [180]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np

# Retail Stores 

In [181]:
Vendors = {'Name':[], 'Logo':[],'Header':[], 'Categories':[],'Location':[], 'Location_1':[] ,'Latest Offers':[], 'Vendor Link':[]}

In [182]:
from selenium.common.exceptions import NoSuchElementException

def find_element_or_none(driver, xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        return None

def get_item(driver, url):
    
    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 
    
    # All info
    a = find_element_or_none(driver, '//*[@id="app"]/div/div/main/div/div[1]/div[1]/div/h3')
    Vendors['Name'].append(a)
    
    b = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/main/div/div[1]/div[1]/div/img')
    b1 = b.get_attribute('src')
    Vendors['Logo'].append(b1)

    c = find_element_or_none(driver, '//*[@id="app"]/div/div/main/div/div[1]/div[1]/div/p')
    Vendors['Header'].append(c)
    
    d = find_element_or_none(driver, '//*[@id="app"]/div/div/main/div/div[1]/div[1]/div/div[1]')
    Vendors['Categories'].append(d)
    
    e = find_element_or_none(driver, '//*[@id="app"]/div/div/main/div/div[1]/div[1]/div/div[2]/div/div[2]/div')
    Vendors['Location'].append(e)

    Vendors['Location_1'].append(None)
        
    f = find_element_or_none(driver, '//*[@id="app"]/div/div/main/div/div[1]/div[2]/div/div/div/div')
    Vendors['Latest Offers'].append(f)
    
    Vendors['Vendor Link'].append(url)

In [183]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
item_links = []
def scrap(src):
    global item_links
    driver = webdriver.Chrome()
    driver.get(src)
    wait = WebDriverWait(driver, 10) 
    time.sleep(3) 

    def click_show_more_button():
        try:
            show_more_button = driver.find_element(By.XPATH, "(//*[contains(concat(' ', @class, ' '), 'v-btn') and contains(concat(' ', @class, ' '), 'v-btn--is-elevated') and contains(concat(' ', @class, ' '), 'v-btn--has-bg') and contains(concat(' ', @class, ' '), 'theme--light') and contains(concat(' ', @class, ' '), 'v-size--default') and contains(concat(' ', @class, ' '), 'primary')])[2]")
            show_more_button.click()
            return True
        except:
            return False

    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            if not click_show_more_button():
                break
        last_height = new_height

    Items_List = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(concat(' ', @class, ' '), 'pl-0') and contains(concat(' ', @class, ' '), 'v-btn') and contains(concat(' ', @class, ' '), 'v-btn--router') and contains(concat(' ', @class, ' '), 'v-btn--text') and contains(concat(' ', @class, ' '), 'theme--light') and contains(concat(' ', @class, ' '), 'v-size--default') and contains(concat(' ', @class, ' '), 'primary--text')]")))

    for item in Items_List:
        item_links.append(item.get_attribute('href'))
    print(len(item_links))
    print(item_links[0])

    for i in item_links:
        max_attempts = 3
        attempts = 0
        while attempts < max_attempts:
            try:
                time.sleep(2)
                print(f"Trying to visit vendor: {i}")
                driver.get(i)
                get_item_info(driver, i)
                break  # Success, no need for further attempts
            except Exception as e:
                print(f"Attempt {attempts+1} failed.")
                attempts += 1
                if attempts == max_attempts:
                    print(f"Failed to visit vendor {i} after {max_attempts} attempts.")
                    break
        
        
    driver.quit()


In [ ]:
url = f'https://www.valu.com.eg/Merchants'
scrap(url)
print(f"Vendors data Loaded Successfully")

In [ ]:
Vendors_data = pd.DataFrame(Vendors)
Vendors_data.head()

In [ ]:
def format_locations(row):
    if pd.notnull(row['Location']):
        locations = row['Location'].split('\n')
        name = row['Name']
        new_rows = []
        for loc in locations:
            location_1 = loc.split(' - ')[-1]  # Remove the vendor name from the location
            location = None
            if len(loc.split(' - ')) >= 3:
                location = loc.split(' - ')[-2]  # Remove the vendor name from the location
            new_rows.append([name, row['Logo'], row['Header'], row['Categories'], location_1, location, row['Latest Offers'],row['Vendor Link']])
        return new_rows
    else:
        # If Locations column is null, create a new row with the null value in Locations
        return [[row['Name'], row['Logo'], row['Header'], row['Categories'], None, None ,row['Latest Offers']]]

In [ ]:
# Apply the function to each row and create a new dataframe
new_rows = Vendors_data.apply(format_locations, axis=1)
Vendors = pd.DataFrame([item for sublist in new_rows for item in sublist], columns=Vendors_data.columns)

In [ ]:
Vendors.head()

In [141]:
Vendors

,Name,Logo,Header,Categories,Location,Location_1,Latest Offers,Vendor Link
0,2B,https://mobagg.valu.com.eg/media-server/vendor...,Laptops - Mobiles & Tablets - Appliances - Ent...,Security Scan\nSharp\nFifa 19\nFilter\nFridge\...,Al Rehab,New Cairo,Offer Expired\nEnds on: 2023-03-01\n0% interes...,https://www.valu.com.eg/vendors/1931
1,2B,https://mobagg.valu.com.eg/media-server/vendor...,Laptops - Mobiles & Tablets - Appliances - Ent...,Security Scan\nSharp\nFifa 19\nFilter\nFridge\...,Technology Mall,Nasr City,Offer Expired\nEnds on: 2023-03-01\n0% interes...,https://www.valu.com.eg/vendors/1931
2,2B,https://mobagg.valu.com.eg/media-server/vendor...,Laptops - Mobiles & Tablets - Appliances - Ent...,Security Scan\nSharp\nFifa 19\nFilter\nFridge\...,El Bostan Mall,Downtown,Offer Expired\nEnds on: 2023-03-01\n0% interes...,https://www.valu.com.eg/vendors/1931
3,2B,https://mobagg.valu.com.eg/media-server/vendor...,Laptops - Mobiles & Tablets - Appliances - Ent...,Security Scan\nSharp\nFifa 19\nFilter\nFridge\...,Galleria Mall,New Cairo,Offer Expired\nEnds on: 2023-03-01\n0% interes...,https://www.valu.com.eg/vendors/1931
4,Al Morshedy,https://mobagg.valu.com.eg/media-server/vendor...,Megastore,Appliances\nBathroom Accessories\nBedroom\nBed...,El Sheikh Zayed,October,None,https://www.valu.com.eg/vendors/2695
5,Al Morshedy,https://mobagg.valu.com.eg/media-server/vendor...,Megastore,Appliances\nBathroom Accessories\nBedroom\nBed...,October,October,None,https://www.valu.com.eg/vendors/2695
6,Carrefour,https://mobagg.valu.com.eg/media-server/vendor...,Megastores,Appliances\nElectronics\nHome\nHome Supplies\n...,City Centre Maadi,Maadi,Offer Expired\nEnds on: 2023-02-21\n24 Months ...,https://www.valu.com.eg/vendors/2797
7,Carrefour,https://mobagg.valu.com.eg/media-server/vendor...,Megastores,Appliances\nElectronics\nHome\nHome Supplies\n...,City Center Alexandria,Alexandria,Offer Expired\nEnds on: 2023-02-21\n24 Months ...,https://www.valu.com.eg/vendors/2797
8,Carrefour,https://mobagg.valu.com.eg/media-server/vendor...,Megastores,Appliances\nElectronics\nHome\nHome Supplies\n...,City Center Almaza,Heliopolis,Offer Expired\nEnds on: 2023-02-21\n24 Months ...,https://www.valu.com.eg/vendors/2797
9,Dejavu,https://mobagg.valu.com.eg/media-server/vendor...,Fashion,Alexandria\nClothes\nFashion\nCairo\nTanta\nHu...,El Batrawy Street,Nasr City,None,https://www.valu.com.eg/vendors/2069


# Saving the data

In [ ]:
from datetime import date
current_date = date.today()
current_date = str(current_date)

Vendors.to_excel(f'Vendors_data_retail_stores_{current_date}.xlsx', index=False)